# 🔄 Notebook: 01 ETL Silver Layer

This notebook handles the **Silver Layer (Data Transformation & Processing)** in the Medallion Architecture of the AI-powered claims pipeline. It focuses on **audio conversion**, **metadata extraction**, and **speech-to-text transcription** using the [OpenAI Whisper model](https://openai.com/index/whisper/).

---

## 🧱 Purpose

To convert raw audio files into a consistent format (MP3), calculate metadata (duration), and transcribe the content into structured text to support downstream AI analytics.



In [0]:
%pip install pydub mutagen openai-whisper numpy>=1.24
dbutils.library.restartPython()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/800.5 kB ? eta -:--:--
     ━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/800.5 kB 5.5 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━ 491.5/800.5 kB 7.5 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 798.7/800.5 kB 8.3 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 7.5 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/194.4 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/65.3 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.

In [0]:
%run "./resources/init" 

DataFrame[]

True

In [0]:
bronze_df = spark.table(f"{CATALOG}.{SCHEMA}.{BRONZE_TABLE}")

meta_table_name = f"{CATALOG}.{SCHEMA}.{META_TABLE}"
if spark._jsparkSession.catalog().tableExists(meta_table_name):
    processed_df = spark.table(meta_table_name).filter("processed = True")
    file_reference_df = bronze_df.join(processed_df, "file_name", "left_anti")
else:
    file_reference_df = bronze_df

if file_reference_df.isEmpty():
    dbutils.notebook.exit("✅ No new files to process. Exiting Silver Layer.")

In [0]:
from pydub import AudioSegment
import os

dbutils.fs.mkdirs(mp3_audio_path)

# Convert each file to mp3 and save to the new volume
for row in file_reference_df.collect():
    file_path = row['file_path']
    try:
        audio = AudioSegment.from_file(file_path)
        new_file_path = os.path.join(mp3_audio_path, os.path.basename(file_path).replace(os.path.splitext(file_path)[1], ".mp3"))
        audio.export(new_file_path, format="mp3")
    except Exception as e:
        print(f"⚠️ Error converting {file_path}: {e}")

In [0]:
from mutagen.mp3 import MP3
from pyspark.sql.types import FloatType
import pyspark.sql.functions as F

mp3_df = (
    spark.createDataFrame(dbutils.fs.ls(mp3_audio_path))
    .withColumn("file_path", F.expr("substring(path, 6, length(path))"))
    .withColumn("file_name", F.expr("substring(name, 1, length(name) - 4)"))
    .filter(F.col("file_name").isin([r["file_name"] for r in file_reference_df.collect()]))
)

def get_audio_duration(file_path):
    try:
        audio = MP3(file_path)
        return float(audio.info.length)
    except Exception as e:
        print(f"⚠️ Error getting duration for {file_path}: {e}")
        return None

duration_udf = F.udf(get_audio_duration, FloatType())
mp3_df = mp3_df.withColumn("audio_duration", F.round(duration_udf("file_path"), 0))

display(mp3_df)

path,name,size,modificationTime,file_path,file_name,audio_duration
dbfs:/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/5e7e3k53_AGT002_2025-01-15 13_35_10.mp3,5e7e3k53_AGT002_2025-01-15 13_35_10.mp3,724461,1747047262000,/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/5e7e3k53_AGT002_2025-01-15 13_35_10.mp3,5e7e3k53_AGT002_2025-01-15 13_35_10,91.0
dbfs:/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/ct4m50n5_AGT005_2025-03-01 12_36_07.mp3,ct4m50n5_AGT005_2025-03-01 12_36_07.mp3,865197,1747047253000,/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/ct4m50n5_AGT005_2025-03-01 12_36_07.mp3,ct4m50n5_AGT005_2025-03-01 12_36_07,108.0
dbfs:/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/nv7032f9_AGT001_2025-02-27 12_40_45.mp3,nv7032f9_AGT001_2025-02-27 12_40_45.mp3,914349,1747047255000,/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/nv7032f9_AGT001_2025-02-27 12_40_45.mp3,nv7032f9_AGT001_2025-02-27 12_40_45,114.0
dbfs:/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/pxvlh18a_AGT001_2025-02-11 11_33_33.mp3,pxvlh18a_AGT001_2025-02-11 11_33_33.mp3,946605,1747047258000,/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/pxvlh18a_AGT001_2025-02-11 11_33_33.mp3,pxvlh18a_AGT001_2025-02-11 11_33_33,118.0
dbfs:/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/ulnocrnh_AGT005_2025-02-04 05_42_51.mp3,ulnocrnh_AGT005_2025-02-04 05_42_51.mp3,956013,1747047260000,/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/ulnocrnh_AGT005_2025-02-04 05_42_51.mp3,ulnocrnh_AGT005_2025-02-04 05_42_51,119.0


In [0]:
from pyspark.sql.functions import regexp_extract

audio_stream_df = (spark.readStream.format("cloudFiles")
.option("cloudFiles.format", "binaryFile")
.option("recursiveFileLookup", "true")
.load(mp3_audio_path))

(audio_stream_df.writeStream
.trigger(availableNow=True)
.option("checkpointLocation", f"/Volumes/{CATALOG}/{SCHEMA}/checkpoints/raw_audio")
.toTable(f"{CATALOG}.{SCHEMA}.raw_audio_binaries")
.awaitTermination())

In [0]:
transcriptions_df = spark.sql(f"""
SELECT
  path as path,
  ai_query(
    'va_whisper_large_v3',
    content,
    failOnError => True
  ) as transcription
FROM `{CATALOG}`.`{SCHEMA}`.`raw_audio_binaries`
""")

# Join the transcriptions back to the original DataFrame
transcribed_df = mp3_df.join(transcriptions_df, on="path", how="inner") \
                                 .select("file_path", "file_name", "transcription", "audio_duration")

In [0]:
display(transcribed_df)

file_path,file_name,transcription,audio_duration
/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/5e7e3k53_AGT002_2025-01-15 13_35_10.mp3,5e7e3k53_AGT002_2025-01-15 13_35_10,"Hello, you've reached out VitalGuard. This is John speaking. How can I assist you today? Hi, this is Sophia Wilson. I was hoping to change my payment method. Of course, Sophia. For security, can I confirm your date of birth and policy number, please? Sure. My date of birth is 15th August 1995 and my policy number is VG924695. Thank you for confirming. What is your current payment method and how would you like to change it? I currently pay by direct debit, but I would like to switch to paying by credit card. Hmm, I can assist you with that. Please note that changing your payment method may affect your policy terms. Would you like me to explain the details? Yes, please do. Let me check that for you. Okay, I've updated your payment method to credit card. Your new payment schedule will be sent to you via email. Also, I'd like to take this opportunity to explain your coverage details. That sounds great, thank you! You're welcome! Just to summarize, I have changed your payment method to credit card and explained your coverage details. If you need further assistance, don't hesitate to reach out and have a wonderful day! Thanks, you too!",91.0
/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/ct4m50n5_AGT005_2025-03-01 12_36_07.mp3,ct4m50n5_AGT005_2025-03-01 12_36_07,"Hello, this is VitalGuard. You're speaking to John. How can I help you today? Hi, this is Emma Johnson. I'm hoping to get some advice on my coverage options. Of course, Emma. For security, can I confirm your date of birth and policy number, please? Sure. My DOB is the 22nd of July, 1990, and my policy number is VG434271. Thank you for confirming. What kind of coverage details are you looking for? I want to know if my policy covers physiotherapy sessions. Okay, let me check that for you. Yes, your current plan includes up to 10 physiotherapy sessions per year, but they need to be prescribed by a GP. That's great to know. Also, if I need to make a complaint about a billing issue, how do I do that? I understand. I will provide you with a complaint reference number and you can submit your complaint through our online portal or over the phone. Here is your reference number. VGCMP4823. Thanks, that helps. Of course, you're very welcome. Is there anything else I can help you with today? No, that's all for now Great Just to summarize, Emma I have confirmed your policy covers physiotherapy with the GP referral and I have provided a complaint reference number for your billing issue If you do need further assistance, don't hesitate to reach out Have a wonderful day Thanks, you too",108.0
/Volumes/samantha_wise/ai_claims_processing_customer_demo/audio_recordings/mp3_audio_recordings/nv7032f9_AGT001_2025-02-27 12_40_45.mp3,nv7032f9_AGT001_2025-02-27 12_40_45,"Hello, thank you for calling VitalGuard. My name is Michelle. How can I assist you today? Hey, this is Liam. Liam Brown. And I was hoping to get a duplicate policy document. Of course Liam. For security, can I confirm your date of birth and policy number? Sure. It's 10th of May 1978 and my policy number is VG8-29910. Thank you for confirming. I understand you're requesting a duplicate policy document. Can you please tell me what happened to the original document? I lost it during the move and now I need it for my records. I apologise for the inconvenience, Liam. I'm going to go ahead and send you a duplicate policy document via email. You should receive it within the next 24 hours. Hmm, okay, that sounds okay. But I'm also a bit frustrated with the premium costs. Can you explain the breakdown? I completely understand your concern, Liam. I'd be happy to provide you with a premium breakdown. Your monthly premium is £3,100, which includes £350 for medical coverage and 

In [0]:
##### For simulated data purposes, not part of final solution

from pyspark.sql.functions import expr, substring, regexp_replace, col, split, to_timestamp, concat_ws

silver_table_path = f"{CATALOG}.{SCHEMA}.{SILVER_TABLE}"

if first_run:
    # Load existing transcriptions from the simulated table
    existing_transcriptions_df = spark.table(f"{CATALOG}.{SCHEMA}.simulated_transcriptions")
    
    # Extract file name from file path
    existing_transcriptions_df = existing_transcriptions_df.withColumn("file_name", expr("substring(file_path, -39, 35)"))
    
    # Combine existing (simulated) transcriptions with new transcriptions
    combined_transcriptions_df = existing_transcriptions_df.unionByName(transcribed_df)
    
    # Extract and transform relevant columns from file name
    combined_transcriptions_df = combined_transcriptions_df.withColumn("file_name", split(col("file_path"), "/").getItem(6)) \
        .withColumn("file_name", regexp_replace(col("file_name"), ".mp3", "")) \
        .withColumn("call_id", split(col("file_name"), "_").getItem(0)) \
        .withColumn("agent_id", split(col("file_name"), "_").getItem(1)) \
        .withColumn("call_datetime", 
            to_timestamp(
                concat_ws(":", split(col("file_name"), "_").getItem(2), 
                split(col("file_name"), "_").getItem(3), 
                split(col("file_name"), "_").getItem(4))))
    
    # Display the combined DataFrame
    display(combined_transcriptions_df)
    
    # Overwrite the transcriptions_silver table with the combined DataFrame
    combined_transcriptions_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(silver_table_path)

else:
    transcribed_df = transcribed_df.withColumn("file_name", regexp_replace(col("file_name"), ".mp3", "")) \
        .withColumn("call_id", split(col("file_name"), "_").getItem(0)) \
        .withColumn("agent_id", split(col("file_name"), "_").getItem(1)) \
        .withColumn("call_datetime", 
            to_timestamp(
                concat_ws(":", split(col("file_name"), "_").getItem(2), 
                split(col("file_name"), "_").getItem(3), 
                split(col("file_name"), "_").getItem(4)))
        )

    display(transcribed_df.select("file_name", "call_id", "agent_id", "call_datetime", "audio_duration", "transcription"))

    if not spark._jsparkSession.catalog().tableExists(silver_table_path):
        transcribed_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(silver_table_path)
    else:
        transcribed_df.write.mode("append").saveAsTable(silver_table_path)

    print(f"✅ Transcriptions written to Silver table: {silver_table_path}")

## ✅ Output
- A clean, enriched Delta table: transcriptions_silver
- Includes transcription text, call metadata, and audio duration for each entry.